In [ ]:
!pip install httpx loguru pandas

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 3.7 MB/s eta 0:00:00


In [ ]:
!pip install httpx loguru pandas

In [ ]:
!pip install httpx[http2] loguru pandas

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.5/57.5 kB 3.9 MB/s eta 0:00:00


In [ ]:
!pip install httpx[http2] loguru pandas nest_asyncio

In [ ]:
import nest_asyncio
import asyncio
import json
import pandas as pd
from typing import List, Dict
from httpx import AsyncClient, Response
from loguru import logger as log

# Aplicar el parche de nest_asyncio
nest_asyncio.apply()

# Inicializar un cliente httpx asíncrono
client = AsyncClient(
    http2=True,
    headers={
        "Accept-Language": "en-US,en;q=0.9",
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/96.0.4664.110 Safari/537.36",
        "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8",
        "Accept-Encoding": "gzip, deflate, br",
        "Cookie": "intl_splash=false"
    },
)


def parse_reviews(response: Response) -> Dict:
    """Analizar los datos de las reseñas de las respuestas de la API de reseñas"""
    data = response.json()
    total_pages = data["totalPages"]
    review_data = data["topics"]
    return {"data": review_data, "total_pages": total_pages}

async def scrape_reviews(skuid: int) -> List[Dict]:
    """Obtener datos de reseñas desde la API de reseñas"""
    first_page = await client.get(f"https://www.bestbuy.com/ugc/v2/reviews?page=1&pageSize=20&sku={skuid}&sort=MOST_RECENT")
    data = parse_reviews(first_page)
    review_data = data["data"]
    total_pages = data["total_pages"]

    log.info(f"Obteniendo paginación de reseñas, {total_pages - 1} páginas más")
    to_scrape = [
        client.get(f"https://www.bestbuy.com/ugc/v2/reviews?page={page_number}&pageSize=20&sku={skuid}&sort=MOST_RECENT")
        for page_number in range(2, total_pages + 1)
    ]
    for response in asyncio.as_completed(to_scrape):
        response = await response
        data = parse_reviews(response)["data"]
        review_data.extend(data)

    log.success(f"Obtenidas {len(review_data)} reseñas de la API de reseñas")
    return review_data

async def run():
    skuid = 6569668  # ID de SKU del producto
    review_data = await scrape_reviews(skuid)

    # Crear un DataFrame de pandas con los datos de las reseñas
    df = pd.DataFrame(review_data)

    # Mostrar el DataFrame
    print(df.head())

    # Almacenar el DataFrame en un archivo JSON
    df.to_json("reviews.json", orient="records", lines=True)

    return df

# Ejecutar la función run
df = asyncio.get_event_loop().run_until_complete(run())

2024-08-16 16:26:45.295 | INFO     | __main__:scrape_reviews:38 - Obteniendo paginación de reseñas, 8 páginas más
2024-08-16 16:26:46.041 | SUCCESS  | __main__:scrape_reviews:48 - Obtenidas 169 reseñas de la API de reseñas


                                     id topicType  rating  recommended  \
0  4ebb42ba-64d6-3f05-b94c-99dfca4abe11    review       5         True   
1  ed62bfd4-9e6f-3371-a6af-823a3d3d9ec5    review       5         True   
2  6c78517f-73e6-3de7-8507-938f3e71e726    review       5         True   
3  a1820743-0bd1-3737-99e9-79e1462bb7a2    review       1         True   
4  0b985652-75be-3a88-8a09-2dd32007eb39    review       5         True   

                                              title  \
0                                    College Laptop   
1                           Fulfilling expectations   
2                                      Great Laptop   
3  Laptop is great but beware of lying sales person   
4                                   This is awesome   

                                                text    author  \
0  Great computer for my new college student.  I ...    Lovely   
1  I like this computer for the moment! I love it...  LucianoR   
2  Love my new laptop, esp

In [ ]:
from google.colab import files
files.download("reviews.json")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
!pip install pyspark


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.3/317.3 MB 4.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.2-py2.py3-none-any.whl size=317812365 sha256=6c3cb81cfb4d74074c5cde4ae502ada17f67c5dd4635cfa4db9f4fed35391558
  Stored in directory: /root/.cache/pip/wheels/34/34/bd/03944534c44b677cd5859f248090daa9fb27b3c8f8e5f49574
Successfully built pyspark


In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col

# Crear una sesión de Spark
spark = SparkSession.builder \
    .appName("Reviews Transformation") \
    .getOrCreate()

# Leer el archivo JSON
df = spark.read.json("reviews.json")

# Seleccionar y renombrar las columnas necesarias
transformed_df = df.select(
    col("id").alias("id"),
    col("text").alias("text"),
    col("rating").alias("rating")
)

# Mostrar el DataFrame transformado
transformed_df.show()

# Guardar el DataFrame en un archivo CSV
transformed_df.write.csv("reviews_transform.csv", header=True)

# Parar la sesión de Spark
spark.stop()


+--------------------+--------------------+------+
|                  id|                text|rating|
+--------------------+--------------------+------+
|4ebb42ba-64d6-3f0...|Great computer fo...|     5|
|ed62bfd4-9e6f-337...|I like this compu...|     5|
|6c78517f-73e6-3de...|Love my new lapto...|     5|
|a1820743-0bd1-373...|Went in to buy th...|     1|
|0b985652-75be-3a8...|I just got this l...|     5|
|a7c80e87-d392-3e8...|This computer is ...|     5|
|8d244806-dcff-3fa...|Would recommend f...|     5|
|a377aac2-5573-39b...|This is a great l...|     4|
|8ab1875c-cc7c-344...|Great computer. W...|     5|
|d9c8f1fb-0596-363...|The product was g...|     1|
|de44c794-777a-308...|Súper good exeede...|     5|
|4b2de625-d06d-3d6...|The computer was ...|     5|
|f730f381-d15f-359...|Excellent service...|     5|
|343f093e-ba59-360...|I love how fast t...|     5|
|ccb07cca-0c35-373...|Overall the lapto...|     4|
|73edea9c-2ff0-3ee...|Always with HP.\n...|     5|
|ecc0ca24-44b1-3d7...|Always en

AnalysisException: [PATH_ALREADY_EXISTS] Path file:/content/reviews_transform.csv already exists. Set mode as "overwrite" to overwrite the existing path.

In [ ]:
import os

# Verificar que el archivo CSV se haya creado
csv_files = [f for f in os.listdir() if f.startswith("reviews_transform")]
print(csv_files)


['reviews_transform.csv']


In [ ]:
import shutil

# Empaquetar los archivos CSV en un solo archivo ZIP para facilitar la descarga
shutil.make_archive("reviews_transform", 'zip', '.', 'reviews_transform.csv')

# Descargar el archivo ZIP
from google.colab import files
files.download("reviews_transform.zip")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>